In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
yelp_dataset_yelp_dataset_path = kagglehub.dataset_download('yelp-dataset/yelp-dataset')

print('Data source import complete.')


Using Colab cache for faster access to the 'yelp-dataset' dataset.
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf
/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json
/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


The Yelp Dataset is a large public resource containing millions of records about businesses, user reviews, customers, and activity on the Yelp platform.

What’s Included:

* Business profiles (details, location, categories, ratings, hours, attributes)

* Reviews (text, stars, date, user/business link, vote counts)

* Users (profiles, review/activity counts, social data)

* Check-ins (visit activity by date)

* Tips (brief user suggestions)

Scope:

* Covers major US and Canadian cities, with data spanning over a decade.

* Contains millions of reviews and hundreds of thousands of businesses.

* Data is semi-structured (JSON), with some nested fields and missing values.

Analytical Value:

* Enables market and competition analysis, sentiment mining, customer trend discovery, and business intelligence for strategy decisions.

* Data must be transformed from JSON to SQL for relational analysis and queries.

This dataset is ideal for exploring real-world business questions, including where and what type of restaurant to open.

**Chunk Size**
We limited the number of rows loaded in the read_json() function so that it would load a small part of the dataset for us to look at.

In [ ]:
# small sample loaded and explored
business_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json'
business_df = pd.read_json(business_path, lines=True, nrows=10)
business_df.info()
business_df.columns
business_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   10 non-null     object 
 1   name          10 non-null     object 
 2   address       10 non-null     object 
 3   city          10 non-null     object 
 4   state         10 non-null     object 
 5   postal_code   10 non-null     int64  
 6   latitude      10 non-null     float64
 7   longitude     10 non-null     float64
 8   stars         10 non-null     float64
 9   review_count  10 non-null     int64  
 10  is_open       10 non-null     int64  
 11  attributes    9 non-null      object 
 12  categories    10 non-null     object 
 13  hours         8 non-null      object 
dtypes: float64(3), int64(3), object(8)
memory usage: 1.2+ KB


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."


**Business Dataset Exploration Summary**

Columns: 14
Key Columns: business_id, name, city, state, stars, categories, attributes
Missing Data:
    attributes (9% missing)
    hours (15% missing)
Geographic Coverage: Multiple cities across the U.S. (e.g., Santa Barbara, Tucson, Philadelphia)

Data Type Notes: Mix of object, float64, and int64 types.

Purpose: Contains core info about businesses — location, rating, and metadata.

Nested dictionaries handling: Business table includes 'attributes' and 'hours' as nested dictionaries. These will need to be flattened or split into separate tables/columns for SQL import since SQL does not support nested structures directly."

In [ ]:
# small sample loaded and explored
checkin_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json'
checkin_df = pd.read_json(checkin_path, lines=True, nrows=10)
checkin_df.info()
checkin_df.columns
checkin_df.head(2)

**Check in table summary: **

Rows explored: 10 (sample)

Columns: 2

Key Columns: business_id, date

Missing Data: None — all values are complete

Data Type Notes: Both columns are of object type; the date column contains multiple check-in timestamps separated by commas for each business

Purpose: Records customer check-in activity for each business. Each row represents one business with all check-in times listed. This data helps analyze foot traffic, peak visit hours, and seasonal trends when linked with the Business table via business_id.

In [ ]:
# small sample loaded and explored
review_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json'
review_df = pd.read_json(review_path, lines=True, nrows=10)
review_df.info()
review_df.columns
review_df.head(2)

**Reviews Dataset Summary **

Rows explored: 10 (sample)

Columns: 9

Key Columns: review_id, user_id, business_id, stars, text, date

Time Range: 2012 onwards

Data Type Notes: Includes object (IDs, text), int64 (ratings and votes), and datetime64[ns] (review date)
Structure: Flat — no nested JSON fields present

Geographic Coverage: Global (primarily U.S., Canada, U.K., and select international cities; determined via business_id link)

Purpose: Captures individual user reviews with ratings, text content, and engagement metrics (useful, funny, cool) for analyzing customer sentiment, business performance, or user behavior trends.

In [ ]:
# small sample loaded and explored
tip_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json'
tip_df = pd.read_json(tip_path, lines=True, nrows=10)
tip_df.info()
tip_df.columns
tip_df.head(2)

**Tip Dataset Exploration Summary**

Rows explored: 10 (sample)

Columns: 5

Key Columns: user_id, business_id, text, date, compliment_count

Missing Data: None (all columns complete)

Time Range: 2012 onwards

Data Type Notes: Includes object, datetime64[ns], and int64 fields

Purpose: Captures user-submitted business reviews with timestamps and compliment counts for sentiment or engagement analysis.

In [ ]:
# small sample loaded and explored
user_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json'
user_df = pd.read_json(user_path, lines=True, nrows=10)
user_df.info()
user_df.columns
user_df.head(2)

**User Dataset Exploration Summary**

Rows explored: 10 (sample)

Columns: 22

Key Columns: user_id, name, review_count, yelping_since, useful, funny, cool, average_stars

Data Type Notes: Mix of object (5), int64 (16), and float64 (1)

Insights: Captures user activity and engagement metrics such as reviews, votes, and compliments. Useful for analyzing user influence, loyalty, and contribution patterns within the Yelp community.

**Azure Database Setup page**
![Screenshot 2025-10-27 at 4.59.38 PM.png](attachment:851a0215-2f57-454a-9fb9-64698ded615a.png)